In [2]:
import pandas as pd 
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
from sklearn.pipeline import Pipeline
import tensorflow as tf
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [3]:
df = pd.read_csv('Churn_Modelling.csv')
data = df.drop(['RowNumber','CustomerId','Surname'],axis =1 )
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[["Geography"]])
onehot_encoder_geo.get_feature_names_out(['Geography'])
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(),columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
data = pd.concat([data.drop('Geography',axis =1 ),geo_encoded_df],axis=1 )
X = data.drop(['Exited'],axis =1 )
y = data['Exited']
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.2,random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

In [15]:
#CV PART 
def create_model(neurons=64, layers=2):
    model = Sequential()
    model.add(Dense(neurons, activation='relu', input_shape=(X_train.shape[1],)))
    for _ in range(layers-1):
        model.add(Dense(neurons, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [14]:

model = KerasClassifier(layers =2 ,neurons = 64 ,build_fn=create_model,verbose=1)



In [16]:
param_grid={
    'neurons':[16,32,64,128],
    'layers':[1,2],
    'epochs':[50,100]
}

In [17]:
grid = GridSearchCV(estimator=model,param_grid=param_grid,n_jobs=-1,cv=3,verbose=1 )
grid_res = grid.fit(X_train,y_train)

print(grid_res.best_score_)
print(grid_res.best_params_)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
Epoch 1/100


c:\Users\Arvind\Desktop\ChurnModelling\venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


250/250 [==============================] - 1s 1ms/step - loss: 0.4863 - accuracy: 0.7895
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4210 - accuracy: 0.8229
Epoch 3/100
250/250 [==============================] - 0s 999us/step - loss: 0.4015 - accuracy: 0.8340
Epoch 4/100
250/250 [==============================] - 0s 996us/step - loss: 0.3836 - accuracy: 0.8451
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3693 - accuracy: 0.8491
Epoch 6/100
250/250 [==============================] - 0s 993us/step - loss: 0.3597 - accuracy: 0.8541
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3535 - accuracy: 0.8547
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3493 - accuracy: 0.8568
Epoch 9/100
250/250 [==============================] - 0s 997us/step - loss: 0.3463 - accuracy: 0.8565
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3445 - accuracy: 